In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import pickle
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [2]:
pick=open('era_predict.pkl','rb')
erareg=pickle.load(pick)

/usr/local/lib/python3.5/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator Lasso from version 0.20.3 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [3]:
def EraPrePro(file):
    current =pd.read_csv(file)
    current=current.rename(columns={'ERA':'ERA_current_year'})
    current=current.reset_index(drop=True)
    for col in current.columns:
        if type(current[col][1])==str:
            current[col]=current[col].str.replace('%','',regex=False)
    numbers=['Age', 'ERA_current_year', 'K/9', 'BB/9',
       'K/BB', 'H/9', 'HR/9', 'WHIP', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%',
       'IFFB%', 'HR/FB', 'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
       'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'K%', 'BB%',
       'Soft%', 'Med%', 'Hard%', 'AVG', 'IFH%', 'BUH%', 'xFIP', 'SIERA',
       'RS/9', 'K-BB%', 'Pull%', 'Cent%', 'Oppo%', 'playerid']
    for col in numbers:
        current[col]=current[col].astype(float)
    return(current)
    

In [4]:
pitchfeatures=['Age', 'ERA_current_year', 'K/9', 'BB/9',
       'K/BB', 'H/9', 'HR/9', 'WHIP', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%',
       'IFFB%', 'HR/FB', 'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
       'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'K%', 'BB%',
       'Soft%', 'Med%', 'Hard%', 'AVG', 'IFH%', 'BUH%','RS/9','Pull%', 'Cent%', 'Oppo%']

In [5]:
current=EraPrePro('pitch2018.csv')


In [6]:
XC=StandardScaler().fit_transform(current[pitchfeatures])
current['preds']=erareg.predict(XC)
preds=current[['Name','ERA_current_year','SIERA','preds']].sort_values('preds')
preds.to_csv('pitching_leaders2018.csv')

ValueError: shapes (232,36) and (35,) not aligned: 36 (dim 1) != 35 (dim 0)

In [7]:
pick=open('woba_predict.pkl','rb')
wobareg=pickle.load(pick)

/usr/local/lib/python3.5/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator Ridge from version 0.20.3 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [13]:
def wobaPrePro(file1,file2,file3):
    current =pd.read_csv(file1)
    current['Name']=' '+current['Name']
    sprint2019=pd.read_csv(file2)
    exitv2019=pd.read_csv(file3)
    sprint2019['Name']=sprint2019[' first_name']+' '+sprint2019['last_name']
    exitv2019['Name']=exitv2019[' first_name']+' '+exitv2019['last_name']
    current=pd.merge(current,sprint2019[['Name','sprint_speed']],on="Name")
    current=pd.merge(current,exitv2019[['avg_hit_angle',
      'anglesweetspotpercent', 'max_hit_speed', 'avg_hit_speed', 'fbld', 'gb',
       'max_distance', 'avg_distance', 'avg_hr_distance','ev95percent', 'brl_percent', 'brl_pa', 'Name']],on="Name")
    current=current.rename(columns={'wOBA':'wOBA_current_year'})
    current=current.reset_index(drop=True)
    for col in current.columns:
        if type(current[col][1])==str:
            current[col]=current[col].str.replace('%','',regex=False)
    numbers=['Age', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP',
       'SLG', 'OPS', 'ISO', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%',
       'HR/FB', 'IFH%', 'BUH%', 'wOBA_current_year', 'O-Swing%', 'Z-Swing%',
       'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%',
       'SwStr%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%', 'sprint_speed','avg_hit_angle',
       'anglesweetspotpercent', 'max_hit_speed', 'avg_hit_speed', 'fbld', 'gb',
       'max_distance', 'avg_distance', 'avg_hr_distance', 'ev95percent',
       'brl_percent', 'brl_pa']
    for col in numbers:
        current[col]=current[col].astype(float)
    current=current.dropna()
    
    
    return(current)

In [23]:
file1,file2,file3='batting2019.csv','sprint_speed2019.csv','exit_velocity2019.csv'
current =pd.read_csv(file1)
current['Name']=' '+current['Name']
sprint2019=pd.read_csv(file2)
exitv2019=pd.read_csv(file3)
sprint2019['Name']=sprint2019[' first_name']+' '+sprint2019['last_name']
exitv2019['Name']=exitv2019[' first_name']+' '+exitv2019['last_name']
current=pd.merge(current,sprint2019[['Name','sprint_speed']],on="Name")
#current=pd.merge(current,exitv2019[['avg_hit_angle',
      #'anglesweetspotpercent', 'max_hit_speed', 'avg_hit_speed', 'fbld', 'gb',
       #'max_distance', 'avg_distance', 'avg_hr_distance','ev95percent', 'brl_percent', 'brl_pa', 'Name']],on="Name")
#current=current.rename(columns={'wOBA':'wOBA_current_year'})
#current=current.reset_index(drop=True)

In [25]:
exitv2019

,last_name,first_name,player_id,attempts,avg_hit_angle,anglesweetspotpercent,max_hit_speed,avg_hit_speed,fbld,gb,max_distance,avg_distance,avg_hr_distance,ev95plus,ev95percent,barrels,brl_percent,brl_pa,Name
0,Cashner,Andrew,488768,381,9.6,31.5,120.6,89.6,94.5,86.5,441,167,413.0,160,42.0,31,8.1,6.1,Andrew Cashner
1,Nova,Ivan,467100,434,9.1,33.4,118.9,88.6,94.0,84.6,431,157,392.0,171,39.7,35,8.1,6.3,Ivan Nova
2,Morin,Mike,573046,88,9.8,38.6,118.3,86.9,91.4,82.3,412,172,392.0,24,27.9,5,5.7,4.6,Mike Morin
3,Cobb,Alex,502171,50,6.4,42.0,118.1,94.9,104.1,87.6,438,184,411.0,30,60.0,13,26.0,21.7,Alex Cobb
4,Samardzija,Jeff,502188,363,14.9,35.8,117.9,89.0,93.2,84.7,440,192,401.0,136,37.6,29,8.0,5.7,Jeff Samardzija
5,Wheeler,Zack,554430,375,11.1,34.1,117.6,86.9,91.3,83.7,447,169,407.0,125,33.3,26,6.9,4.7,Zack Wheeler
6,Stripling,Ross,548389,220,10.3,23.2,117.4,88.1,93.8,85.7,415,156,401.0,85,38.6,16,7.3,5.0,Ross Stripling
7,Chatwood,Tyler,543022,150,4.6,35.3,117.0,87.5,90.4,86.3,418,145,389.0,54,36.0,4,2.7,1.8,Tyler Chatwood
8,Alexander,Tyler,641302,73,14.5,30.1,116.8,90.3,96.1,84.1,426,186,387.0,30,41.1,5,6.8,5.3,Tyler Alexander
9,Kolarek,Adam,592473,131,0.7,24.4,116.8,87.8,92.9,85.8,402,125,378.0,49,37.4,9,6.9,4.9,Adam Kolarek


In [14]:
current=wobaPrePro('batting2019.csv','sprint_speed2019.csv','exit_velocity2019.csv')

IndexError: index out of bounds

In [10]:
wobafeatures=['Age', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP',
       'SLG', 'ISO', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%',
       'HR/FB', 'IFH%', 'BUH%', 'wOBA_current_year', 'O-Swing%', 'Z-Swing%',
       'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%',
       'SwStr%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%', 'sprint_speed', 'avg_hit_angle',
       'anglesweetspotpercent', 'max_hit_speed', 'avg_hit_speed', 'fbld', 'gb',
       'max_distance', 'avg_distance', 'avg_hr_distance', 'ev95percent',
       'brl_percent', 'brl_pa']

In [11]:
XC=StandardScaler().fit_transform(current[wobafeatures])
current['preds']=wobareg.predict(XC)
preds=current[['Name','wOBA_current_year','preds']].sort_values('preds',ascending=False)
preds.to_csv('batting_leaders.csv')